In [2]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler
scaler = MinMaxScaler(feature_range=(0, 1))
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, GRU
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

class MyGRU (Sequential):
    def __init__(self, look_back, dense_units =[],unit=64, optimizer='adam',name='gru'):
        super().__init__(name=name)
        self.look_back = look_back
        self.add(GRU(units=unit, activation='relu', input_shape=(look_back,5)))
        for unit in dense_units:
            self.add(Dense(units=unit, activation='relu'))
        self.add(Dense(units=5, activation='sigmoid' ))
        self.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
        # self.trainX, self.trainY, self.testX, self.testY = self.prepare_data(series, test_size, look_back, scaler, is_ema=True)
    # def evaluate(self, X, y):


# class MyGridSearchCV(GridSearchCV):
#     def __init__(self, look_back, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1, refit=True, return_train_score=False):
#         estimator = MyGRU(look_back)
#         super().__init__(
#             estimator=estimator,
#             param_grid=param_grid,
#             scoring=scoring,
#             cv=cv,
#             n_jobs=n_jobs,
#             verbose=verbose,
#             refit=refit,
#             return_train_score=return_train_score
#         )



In [3]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

def exponential_moving_average(data, span):
    return data.ewm(span=span, adjust=False).mean()

def read_data(file_path):
    from pandas import read_csv
    series_influ_A_df = read_csv(file_path, index_col=0, engine='python')
    series_influ_A_df = series_influ_A_df.rename(columns= {"Influenza A - All types of surveillance": "case"})
    series_influ_A_df = series_influ_A_df[["case", "temp", "tempmax", "dew","windspeed"]]
    return series_influ_A_df.dropna()

def prepare_data(series, n_test, look_back, scaler, is_ema = False):
    if is_ema:
        span = 52  # Bạn có thể điều chỉnh độ dài span tùy ý
        series['case'] = exponential_moving_average(series['case'], span)
    series = series.astype('float32')
    series = series.values
    dataset = scaler.fit_transform(series)
    train = dataset[:-n_test, :]
    test = dataset[-n_test-look_back:, :]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    return trainX, trainY, testX, testY

def forecast(input, model):
    input = input.reshape(1, input.shape[0], input.shape[1])
    predicted = model.predict(input, verbose=0)
    return predicted[0]

def make_forecasts(test, model):
    forecasts = []
    inputStart = test[0]
    inputX = inputStart.reshape(1, inputStart.shape[0], inputStart.shape[1])
    for i in range(len(test)):
        predicted = forecast(inputX[i], model)
        forecasts.append(predicted)
        inputXContinue = np.vstack((inputX[-1][1:], predicted))
        inputX = np.append(inputX, [inputXContinue], axis=0)
    
    return forecasts

In [4]:
df = read_data('../temp_data/influA_vietnam_last_10_days.csv')

In [12]:
import itertools
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

def LSTM_HyperParameter_Tuning(config, df, scaler):
    
    n_neurons, n_batch_sizes, dropouts, look_backs = config
    possible_combinations = list(itertools.product(n_neurons, n_batch_sizes, dropouts, look_backs))
    
    print(possible_combinations)
    print('\n')
    
    hist = []
    for i in range(0, len(possible_combinations)):
        print(f'{i+1}th combination: \n')
        print('--------------------------------------------------------------------')
        n_neurons, n_batch_size, dropout, look_back = possible_combinations[i]

        # model = Sequential()

        # model.add(GRU(units=n_neurons[0], input_shape=(look_back, 5)))

        # for i in range(1, len(n_neurons)):
        #     model.add(Dense(units=n_neurons[i], activation='sigmoid' ))
        
        # model.add(Dense(units=5, activation='sigmoid' ))

        # model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

        trainX, trainY, testX, testY = prepare_data(df, 50, look_back, scaler, is_ema=True)
        model = MyGRU(look_back=look_back, dense_units=n_neurons[1:], unit=n_neurons[0])

        es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
        '''''
        From the mentioned article above --> If a validation dataset is specified to the fit() function via the validation_data or v
        alidation_split arguments,then the loss on the validation dataset will be made available via the name “val_loss.”
        '''''
        file_path = '../model/best_gru_model.keras'

        mc = ModelCheckpoint(file_path, monitor='loss', mode='min', verbose=1, save_best_only=True)

        '''''
        cb = Callback(...)  # First, callbacks must be instantiated.
        cb_list = [cb, ...]  # Then, one or more callbacks that you intend to use must be added to a Python list.
        model.fit(..., callbacks=cb_list)  # Finally, the list of callbacks is provided to the callback argument when fitting the model.
        '''''

        model.fit(trainX, trainY,batch_size=n_batch_size, callbacks=[es, mc], verbose=0, epochs=100)
        train_accuracy = model.evaluate(trainX, trainY, verbose=0)
        test_accuracy = model.evaluate(testX, testY, verbose=0)
        hist.append(list((n_neurons, n_batch_size, dropout,look_back,
                          train_accuracy, test_accuracy)))
        # print(f'{str(i)}-th combination = {possible_combinations[i]} \n train accuracy: {train_accuracy}')
        
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
         
    return hist

In [14]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler
scaler = MinMaxScaler(feature_range=(0, 1))
# convert an array of values into a dataset matrix
config = [[[64,32,16], [32,16], [64], [32]], [8, 16, 32], [0.2],[12,15,17]]  
hist = LSTM_HyperParameter_Tuning(config, df, scaler)

[([64, 32, 16], 8, 0.2, 12), ([64, 32, 16], 8, 0.2, 15), ([64, 32, 16], 8, 0.2, 17), ([64, 32, 16], 16, 0.2, 12), ([64, 32, 16], 16, 0.2, 15), ([64, 32, 16], 16, 0.2, 17), ([64, 32, 16], 32, 0.2, 12), ([64, 32, 16], 32, 0.2, 15), ([64, 32, 16], 32, 0.2, 17), ([32, 16], 8, 0.2, 12), ([32, 16], 8, 0.2, 15), ([32, 16], 8, 0.2, 17), ([32, 16], 16, 0.2, 12), ([32, 16], 16, 0.2, 15), ([32, 16], 16, 0.2, 17), ([32, 16], 32, 0.2, 12), ([32, 16], 32, 0.2, 15), ([32, 16], 32, 0.2, 17), ([64], 8, 0.2, 12), ([64], 8, 0.2, 15), ([64], 8, 0.2, 17), ([64], 16, 0.2, 12), ([64], 16, 0.2, 15), ([64], 16, 0.2, 17), ([64], 32, 0.2, 12), ([64], 32, 0.2, 15), ([64], 32, 0.2, 17), ([32], 8, 0.2, 12), ([32], 8, 0.2, 15), ([32], 8, 0.2, 17), ([32], 16, 0.2, 12), ([32], 16, 0.2, 15), ([32], 16, 0.2, 17), ([32], 32, 0.2, 12), ([32], 32, 0.2, 15), ([32], 32, 0.2, 17)]


1th combination: 

--------------------------------------------------------------------


d:\my_study\gr3\DATN\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: loss improved from inf to 0.04389, saving model to ../model/best_gru_model.keras

Epoch 2: loss improved from 0.04389 to 0.02190, saving model to ../model/best_gru_model.keras

Epoch 3: loss improved from 0.02190 to 0.01587, saving model to ../model/best_gru_model.keras

Epoch 4: loss improved from 0.01587 to 0.01325, saving model to ../model/best_gru_model.keras

Epoch 5: loss improved from 0.01325 to 0.01120, saving model to ../model/best_gru_model.keras

Epoch 6: loss improved from 0.01120 to 0.01016, saving model to ../model/best_gru_model.keras

Epoch 7: loss improved from 0.01016 to 0.00996, saving model to ../model/best_gru_model.keras

Epoch 8: loss improved from 0.00996 to 0.00980, saving model to ../model/best_gru_model.keras

Epoch 9: loss improved from 0.00980 to 0.00949, saving model to ../model/best_gru_model.keras

Epoch 10: loss improved from 0.00949 to 0.00934, saving model to ../model/best_gru_model.keras

Epoch 11: loss improved from 0.00934 to 0.00897, sav

d:\my_study\gr3\DATN\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: loss improved from inf to 0.04759, saving model to ../model/best_gru_model.keras

Epoch 2: loss improved from 0.04759 to 0.02123, saving model to ../model/best_gru_model.keras

Epoch 3: loss improved from 0.02123 to 0.01296, saving model to ../model/best_gru_model.keras

Epoch 4: loss improved from 0.01296 to 0.01041, saving model to ../model/best_gru_model.keras

Epoch 5: loss improved from 0.01041 to 0.00963, saving model to ../model/best_gru_model.keras

Epoch 6: loss improved from 0.00963 to 0.00930, saving model to ../model/best_gru_model.keras

Epoch 7: loss improved from 0.00930 to 0.00904, saving model to ../model/best_gru_model.keras

Epoch 8: loss improved from 0.00904 to 0.00885, saving model to ../model/best_gru_model.keras

Epoch 9: loss did not improve from 0.00885

Epoch 10: loss improved from 0.00885 to 0.00851, saving model to ../model/best_gru_model.keras

Epoch 11: loss improved from 0.00851 to 0.00832, saving model to ../model/best_gru_model.keras

Epoch 1

...


Epoch 1: loss improved from inf to 0.08854, saving model to ../model/best_gru_model.keras

Epoch 2: loss improved from 0.08854 to 0.06552, saving model to ../model/best_gru_model.keras

Epoch 3: loss improved from 0.06552 to 0.04109, saving model to ../model/best_gru_model.keras

Epoch 4: loss improved from 0.04109 to 0.03012, saving model to ../model/best_gru_model.keras

Epoch 5: loss improved from 0.03012 to 0.02437, saving model to ../model/best_gru_model.keras

Epoch 6: loss improved from 0.02437 to 0.01989, saving model to ../model/best_gru_model.keras

Epoch 7: loss improved from 0.01989 to 0.01729, saving model to ../model/best_gru_model.keras

Epoch 8: loss improved from 0.01729 to 0.01556, saving model to ../model/best_gru_model.keras

Epoch 9: loss improved from 0.01556 to 0.01412, saving model to ../model/best_gru_model.keras

Epoch 10: loss improved from 0.01412 to 0.01263, saving model to ../model/best_gru_model.keras

Epoch 11: loss improved from 0.01263 to 0.01153, sav

d:\my_study\gr3\DATN\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: loss improved from inf to 0.09287, saving model to ../model/best_gru_model.keras

Epoch 2: loss improved from 0.09287 to 0.07017, saving model to ../model/best_gru_model.keras

Epoch 3: loss improved from 0.07017 to 0.04302, saving model to ../model/best_gru_model.keras

Epoch 4: loss improved from 0.04302 to 0.03110, saving model to ../model/best_gru_model.keras

Epoch 5: loss improved from 0.03110 to 0.02726, saving model to ../model/best_gru_model.keras

Epoch 6: loss improved from 0.02726 to 0.02205, saving model to ../model/best_gru_model.keras

Epoch 7: loss improved from 0.02205 to 0.01678, saving model to ../model/best_gru_model.keras

Epoch 8: loss improved from 0.01678 to 0.01401, saving model to ../model/best_gru_model.keras

Epoch 9: loss improved from 0.01401 to 0.01237, saving model to ../model/best_gru_model.keras

Epoch 10: loss improved from 0.01237 to 0.01180, saving model to ../model/best_gru_model.keras

Epoch 11: loss improved from 0.01180 to 0.01101, sav

d:\my_study\gr3\DATN\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1: loss improved from inf to 0.07347, saving model to ../model/best_gru_model.keras

Epoch 2: loss improved from 0.07347 to 0.04764, saving model to ../model/best_gru_model.keras

Epoch 3: loss improved from 0.04764 to 0.03310, saving model to ../model/best_gru_model.keras

Epoch 4: loss improved from 0.03310 to 0.02692, saving model to ../model/best_gru_model.keras

Epoch 5: loss improved from 0.02692 to 0.02118, saving model to ../model/best_gru_model.keras

Epoch 6: loss improved from 0.02118 to 0.01712, saving model to ../model/best_gru_model.keras

Epoch 7: loss improved from 0.01712 to 0.01440, saving model to ../model/best_gru_model.keras

Epoch 8: loss improved from 0.01440 to 0.01214, saving model to ../model/best_gru_model.keras

Epoch 9: loss improved from 0.01214 to 0.01151, saving model to ../model/best_gru_model.keras

Epoch 10: loss improved from 0.01151 to 0.01109, saving model to ../model/best_gru_model.keras

Epoch 11: loss improved from 0.01109 to 0.01056, sav

In [17]:
import pandas as pd
hist = pd.DataFrame(hist)
hist = hist.sort_values(by=[4], ascending=True)
hist

,0,1,2,3,4,5
2,"[64, 32, 16]",8,0.2,17,"[0.006289479322731495, 0.0793062373995781]","[0.042470987886190414, 0.20608490705490112]"
1,"[64, 32, 16]",8,0.2,15,"[0.006782494485378265, 0.08235590159893036]","[0.04181318357586861, 0.20448271930217743]"
19,[64],8,0.2,15,"[0.006855102721601725, 0.08279554545879364]","[0.008570901118218899, 0.09257916361093521]"
32,[32],16,0.2,17,"[0.006969427689909935, 0.08348309993743896]","[0.008629870600998402, 0.09289709478616714]"
23,[64],16,0.2,17,"[0.00697379931807518, 0.08350927382707596]","[0.008660294115543365, 0.093060702085495]"
31,[32],16,0.2,15,"[0.006997907999902964, 0.08365350216627121]","[0.008965819142758846, 0.09468801319599152]"
0,"[64, 32, 16]",8,0.2,12,"[0.007063673343509436, 0.0840456634759903]","[0.047422874718904495, 0.21776792407035828]"
13,"[32, 16]",16,0.2,15,"[0.007107175886631012, 0.08430406451225281]","[0.008762940764427185, 0.09361057728528976]"
18,[64],8,0.2,12,"[0.007180424407124519, 0.0847373828291893]","[0.00906034093350172, 0.09518582373857498]"
22,[64],16,0.2,15,"[0.007186472415924072, 0.08477306365966797]","[0.009039727970957756, 0.09507748484611511]"


In [19]:
print(f'Best Combination: \n units = {hist.iloc[0, 0]}\n n_batch_size = {hist.iloc[0, 1]}\n drop_out = {hist.iloc[0, 2]}\n look_back = {hist.iloc[0, 3]}\n')
print('**************************')
# print(f'Results Before Tunning:\n Test Set RMSE: {np.round(results, 4)[1]}\n')
print(f'Results After Tunning:\n Test Set RMSE: {np.round(hist.iloc[0, -1], 4)[1]}\n')
# print(f'{np.round((results[1] - hist.iloc[0, -1][1])*100/np.round(results, 4)[1])}% Improvement')


Best Combination: 
 units = [64, 32, 16]
 n_batch_size = 8
 drop_out = 0.2
 look_back = 17

**************************
Results After Tunning:
 Test Set RMSE: 0.2061

